In [17]:
import numpy as np
import matplotlib.pyplot as plt
import random
import sys
import os
import scipy
import math
from sklearn.metrics import r2_score
from pathlib import Path
import subprocess

from Bio.SeqIO import read, parse
import orthoani

from multiprocess import Pool

max_pool = 16
lim_file = 100

sys.path.append('../')
sys.path.append('../../')


In [2]:
from db_sketching.kmer_set import KMerSet, FracMinHash, TruncatedKMerSet

In [3]:
def cond(kmer_hash):
    hash = (976369 * kmer_hash + 1982627) % 10000
    if hash < 50:
        return True
    else:
        return False
    
random.seed(10)

mod = 1000000007
r0 = random.randint(0,1000000006)
r1 = random.randint(0,1000000006)
r2 = random.randint(0,1000000006)
r3 = random.randint(0,1000000006)
c = 200

def poly_cond(kmer_hash):
    hash_mod = kmer_hash % mod
    hash = (r0 + hash_mod * (r1 + hash_mod * (r2 + hash_mod * (r3)))) % mod
    return hash < (mod / c)

def all_cond(kmer_hash):
    return True

In [4]:
def compare_ANI(genome_file_1, genome_file_2, kmer_length, kmer_class, sketching_condition):
    """
    Function to compute the estimated ANI using FracMinHash 
    and compare it to the value from OrthoANI
    """
    return compute_ortho_ani(genome_file_1, genome_file_2), compute_kmer_ani(genome_file_1, genome_file_2, kmer_length, kmer_class, sketching_condition)

def compute_kmer_sketches(genome_file, kmer_class, sketching_condition, kmer_length, canonical):
    genome_kmer = kmer_class(sketching_condition, kmer_length, canonical)
    genome_kmer.insert_file(genome_file)
    return genome_kmer

def compute_kmer_sketches_parallel(genome_files, kmer_length, kmer_class, sketching_cond, canonical):
    args = [(g,kmer_class,sketching_cond,kmer_length,canonical) for g in genome_files]
    with Pool(max_pool) as p:
        return p.starmap(compute_kmer_sketches,args)


def compute_kmer_ani(genome_1_kmer, genome_2_kmer):    
    kmer_estimated_ani = genome_1_kmer.ANI_estimation(genome_2_kmer)
    return kmer_estimated_ani


def compute_kmer_ani_parallel(kmer_sketches_1,kmer_sketches_2):
    args = [(s1,s2) for s1,s2 in zip(kmer_sketches_1,kmer_sketches_2)]
    with Pool(max_pool) as p:
        return p.starmap(compute_kmer_ani,args)

def compute_ortho_ani(genome_file_1, genome_file_2):
    try:
        genome_1_read = parse(genome_file_1,"fasta")
        genome_2_read = parse(genome_file_2,"fasta")
        ortho_ani_value = orthoani.orthoani(genome_1_read,genome_2_read)

        return ortho_ani_value
    except:
        return 0




In [5]:
# analysis_type = "Single-Species"
# analysis_data_dir = "Escherichia coli" 

analysis_type = "Single-Genus"
analysis_data_dir = "Yersinia" 

In [6]:
data_dir = os.path.join(os.path.join("../../data_temp",analysis_type),analysis_data_dir)
genome_files = []


for filename in os.listdir(data_dir):
    full_filename = os.path.join(data_dir,filename)
    genome_files.append(full_filename)


print(genome_files)
print(len(genome_files))

['../../data_temp/Single-Genus/Yersinia/64.fna', '../../data_temp/Single-Genus/Yersinia/32.fna', '../../data_temp/Single-Genus/Yersinia/99.fna', '../../data_temp/Single-Genus/Yersinia/43.fna', '../../data_temp/Single-Genus/Yersinia/7.fna', '../../data_temp/Single-Genus/Yersinia/98.fna', '../../data_temp/Single-Genus/Yersinia/58.fna', '../../data_temp/Single-Genus/Yersinia/90.fna', '../../data_temp/Single-Genus/Yersinia/14.fna', '../../data_temp/Single-Genus/Yersinia/41.fna', '../../data_temp/Single-Genus/Yersinia/67.fna', '../../data_temp/Single-Genus/Yersinia/73.fna', '../../data_temp/Single-Genus/Yersinia/17.fna', '../../data_temp/Single-Genus/Yersinia/56.fna', '../../data_temp/Single-Genus/Yersinia/50.fna', '../../data_temp/Single-Genus/Yersinia/2.fna', '../../data_temp/Single-Genus/Yersinia/46.fna', '../../data_temp/Single-Genus/Yersinia/52.fna', '../../data_temp/Single-Genus/Yersinia/65.fna', '../../data_temp/Single-Genus/Yersinia/95.fna', '../../data_temp/Single-Genus/Yersinia/45

In [7]:
checked_genome_files = []
for file in genome_files:
    try:
        parsed_file = parse(file,"fasta")
        assert(len([record for record in parsed_file]) > 0)
        checked_genome_files.append(file)
    except:
        print(f"File {file} is damaged / invalid")


In [8]:
print(checked_genome_files)
print(len(checked_genome_files))

['../../data_temp/Single-Genus/Yersinia/64.fna', '../../data_temp/Single-Genus/Yersinia/32.fna', '../../data_temp/Single-Genus/Yersinia/99.fna', '../../data_temp/Single-Genus/Yersinia/43.fna', '../../data_temp/Single-Genus/Yersinia/7.fna', '../../data_temp/Single-Genus/Yersinia/98.fna', '../../data_temp/Single-Genus/Yersinia/58.fna', '../../data_temp/Single-Genus/Yersinia/90.fna', '../../data_temp/Single-Genus/Yersinia/14.fna', '../../data_temp/Single-Genus/Yersinia/41.fna', '../../data_temp/Single-Genus/Yersinia/67.fna', '../../data_temp/Single-Genus/Yersinia/73.fna', '../../data_temp/Single-Genus/Yersinia/17.fna', '../../data_temp/Single-Genus/Yersinia/56.fna', '../../data_temp/Single-Genus/Yersinia/50.fna', '../../data_temp/Single-Genus/Yersinia/2.fna', '../../data_temp/Single-Genus/Yersinia/46.fna', '../../data_temp/Single-Genus/Yersinia/52.fna', '../../data_temp/Single-Genus/Yersinia/65.fna', '../../data_temp/Single-Genus/Yersinia/95.fna', '../../data_temp/Single-Genus/Yersinia/45

In [9]:
checked_genome_files = checked_genome_files[:lim_file]

In [10]:

def get_genome_length(genome_file):
    genome_kmer = KMerSet(20)
    genome_kmer.insert_file(genome_file)
    return genome_kmer.length

def compute_length_parallel(genome_files):
    with Pool(max_pool) as p:
        return p.starmap(get_genome_length,([(g_file,) for g_file in genome_files]))

    
checked_genome_lengths = compute_length_parallel(checked_genome_files)
print(checked_genome_lengths)


[4563812, 4946280, 4711329, 3743544, 4618658, 4679146, 4719608, 4472252, 4588962, 4619578, 4719102, 4585491, 4520847, 4770670, 4598307, 4656598, 4590473, 4395097, 4464171, 4592664, 4616084, 4845299, 4807288, 4884396, 4604287, 4499457, 4641984, 3825640, 4725862, 3761178, 4933457, 4837316, 4817417, 4511635, 4663094, 4551793, 4827235, 3854074, 4572981, 4386001, 4560303, 4745632, 4422732, 4711317, 4710674, 4847790, 4777134, 4608546, 4613199, 4565715, 4623424, 4389464, 4407320, 4817654, 4650569, 5098804, 4681481, 4476549, 4637667, 4726818, 4913347, 4607986, 4263065, 4669155, 4928910, 4787503, 3895501, 4590539, 3847543, 4423715, 4554479, 4573297, 4383307, 4732312, 4568487, 4511449, 4673139, 4536232, 4486116, 4928829, 4838262, 4509900, 4624562, 4804290, 4444328, 4617115, 4929654, 4509471, 4633803, 4602055, 4553897, 4928916, 4262011, 4408836, 4616215, 4659915, 4873117, 4645798, 4553687, 4503787]


In [11]:
ortho_vals = [] 

def compute_ortho_ani_parallel(genome_files_1,genome_files_2):
    args = [(g1,g2) for g1,g2 in zip(genome_files_1,genome_files_2)]
    with Pool(max_pool) as p:
        return p.starmap(compute_ortho_ani,args)

def compute_pairwise_ortho(genome_files):
    genome_files_1, genome_files_2 = zip(*[(g1,g2) for g1 in genome_files for g2 in genome_files])
    return compute_ortho_ani_parallel(genome_files_1,genome_files_2)

# ortho_vals = compute_pairwise_ortho(checked_genome_files)
ortho_vals = compute_ortho_ani_parallel(checked_genome_files,checked_genome_files[1:])
print(ortho_vals)
print(len(ortho_vals))

[0.8155060721757322, 0.8155629661590524, 0.7775694609277058, 0.7770112703379224, 0.8139086520655271, 0.8145762059669303, 0.8182484863038064, 0.9590551359275054, 0.814788984620762, 0.816687275394206, 0.8168345854638422, 0.8133865853227232, 0.8141688566210046, 0.9986249891618497, 0.9986158534602879, 0.819580976618705, 0.8198445720371805, 0.8164481490015361, 0.9592696641988366, 0.814606230116649, 0.8137981275862068, 0.8135348383212935, 0.9982575888767721, 0.9990654644495413, 0.9979634064362336, 0.9980883803986711, 0.7777042514595496, 0.7788261273885351, 0.7785007220594391, 0.7800308965383119, 0.8389320351180196, 0.8484990568150014, 0.8479088746766312, 0.8163959944173064, 0.820104688689035, 0.8202258996415771, 0.7779980613997424, 0.7789688865278368, 0.8141239119451461, 0.8130341357142857, 0.8136915267041461, 0.8145850198915009, 0.8208285530246453, 0.8373335814439141, 0.8477068355855856, 0.8167722886452069, 0.9981965683085502, 0.9958503503480278, 0.8137803866619369, 0.8141035752401281, 0.99

In [12]:
# ortho_vals = [0.7985283532392328, 0.7649304964871194, 0.6295170786516854, 0.6265655648535565, 0.7939133227315248, 0.8039446536951089, 0.9932520136208853, 0.992398422705856, 0.9792053953223047, 0.6263259615384615, 0.6265060655737705, 0.9924258213601532, 0.7469532214895268, 0.756656126340882, 0.8038214523200584, 0.8044412653651483, 0.678072655882353, 0.6794797867170627, 0.6761155484247374, 0.6805184098837209, 0.6533521353670162, 0.6525219398642095, 0.8026840069622573, 0.984091539213291, 0.9934329466357309, 0.9833911382901429, 0.774011161395856, 0.6796020151811949, 0.6802019305794607, 0.9062559435510338, 0.8037254539032144, 0.984175178364605, 0.6566031484716157, 0.6567052230971129, 0.985175789536408, 0.8029902254398827, 0.678352203967762, 0.6839517082484725, 0.6493673401297497, 0.6514046015180266, 0.8044338989825581, 0.804979523297491, 0.8049422952002887, 0.9795228456474031, 0.98075642149579, 0.9105555241818708, 0.9102294753173483, 0.8035561884816754, 0.8035275505226481, 0.9111223991935484, 0.6550677155555555, 0.6570620596085409, 0.9825480234005477, 0.9953008255378859, 0.9953008255378859, 0.6566498260481712, 0.6555431781557743, 0.8076207707653247, 0.8046716394895427, 0.9837521279867518, 0.9935444431527551, 0.9940015730472517, 0.9937200609756097, 0.8069735916249106, 0.917811760212554, 0.8040779249644381, 0.9828230479370379, 0.8034352319970305, 0.9822912388701259, 0.8045523905429072, 0.9873331730304495, 0.8038084274919035, 0.8742895960811276, 0.8009336061870004, 0.8888446614127262, 0.8039920773461061, 0.8096178063943161, 0.9087031710111716, 0.802961243627094, 0.7935492274272762, 0.7779480686309738, 0.6537470446265938, 0.649564287099903, 0.9976547598944591, 0.8036643078586878, 0.8015705996503496]

In [13]:
kmer_class = FracMinHash
canonical = True

In [14]:


kmer_sketches = [[]] * 40
kmer_vals = [[]] * 40
for kmer_length in range(10,39+1):
    kmer_sketches[kmer_length] = compute_kmer_sketches_parallel(checked_genome_files, kmer_length, kmer_class, poly_cond, canonical=canonical)
    # kmer_sketches_1,kmer_sketches_2 = zip(*[(s1,s2) for s1 in kmer_sketches[kmer_length] for s2 in kmer_sketches[kmer_length]])
    kmer_sketches_1,kmer_sketches_2 = zip(*[(s1,s2) for s1,s2 in zip(kmer_sketches[kmer_length],kmer_sketches[kmer_length][1:])])
    kmer_vals[kmer_length] = compute_kmer_ani_parallel(kmer_sketches_1,kmer_sketches_2)
    print(len(kmer_vals[kmer_length]))
    



print(kmer_vals)

99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
[[], [], [], [], [], [], [], [], [], [], [0.997315057947169, 0.9976897273279198, 0.9956839015777093, 0.9975215595058661, 0.9960711904120717, 0.9974015419185777, 0.9960262698421642, 0.9983193229798263, 0.9972145343362916, 0.9964454895354798, 0.9974100881433595, 0.9963178529221762, 0.9972885836408604, 0.9999196496682925, 0.9998794527008747, 0.9969425948160908, 0.9972996777943054, 0.9958559069909891, 0.9982345120724002, 0.997211106895739, 0.9967357387212784, 0.9970179374814666, 0.9999597998024831, 0.9998794527008747, 0.9996375275475866, 1.0, 0.9959911235945393, 0.9973576215251274, 0.9959845523374311, 0.9973993965418172, 0.9973139651777594, 0.9971444129788104, 0.9963133274566739, 0.9971637483024632, 0.9959494599130446, 0.9973993965418172, 0.9958626981984697, 0.9974828993635523, 0.995909454712379, 0.9971149881070888, 0.9966528879839409, 0.9969250441972027, 0.9969728145404423, 0.9964682435311135, 0.9974

In [19]:
Path(f"../../plots/{analysis_type}-{analysis_data_dir}/").mkdir(parents=True, exist_ok=True)

for kmer_length in range(10,39+1):
    fig = plt.figure(figsize=(10,10),dpi=300)
    ax = fig.add_subplot()
    filter_o, filter_k = zip(*[(o,k) for o,k in zip(ortho_vals,kmer_vals[kmer_length]) if (k > 0 and o > 0)])
    lin_fit = np.poly1d(np.polyfit(filter_o, filter_k, 1))
    ax.plot(filter_o,filter_o,"r-")
    ax.scatter(filter_o,filter_k,marker="^",s=[2 for _ in filter_o])
    # ax.scatter(filter_o,filter_o,marker="o",s=[2 for _ in filter_o])
    ax.plot(np.unique(filter_o), lin_fit(np.unique(filter_o)))
    plt.figtext(0.7,0.15,f"Zeros dropped : {len(ortho_vals) - len(filter_o)}")
    pearson_coeff = scipy.stats.pearsonr(filter_o,filter_k)

    plt.figtext(0.7,0.17,f"Pearson Corr. Coeff.: {pearson_coeff.statistic:.3f}")
    plt.figtext(0.7,0.19,f"Pearson Corr. p.: {pearson_coeff.pvalue:.3e}")

    ax.legend(["OrthoANI",f"{kmer_length}-mer dots",f"{kmer_length}-mer line : {lin_fit}"])
    plt.plot()
    plt.savefig(f"../../plots/{analysis_type}-{analysis_data_dir}/{analysis_type}-{analysis_data_dir}-{len(ortho_vals)}genomes-{kmer_length}mer-{c=}-{canonical=}-estimated-ANI.png")
    plt.close()

    

